In [ ]:
import os
import sys
from os.path import join

import open3d

sys.path.append(os.getcwd())

from helpers.sensors import *
from helpers.visualization import *

def mask_points_in_bbox(bbox_coords, valid_pts):
    # Returns all points falling within bbox corners

    # For each bounding box, retrieve points in the box
    bbox_pts_all_mask = np.zeros((valid_pts.shape[0],))
    for obj_idx in range(0, bbox_coords.shape[0]):
        xmin, xmax = bbox_coords[obj_idx, 0],  bbox_coords[obj_idx, 2]
        ymin, ymax = bbox_coords[obj_idx, 1],  bbox_coords[obj_idx, 3]
        if xmin==0 and xmax==0 and ymin==0 and ymax==0:
            continue

        bbox_point_mask = np.logical_and(
            np.logical_and(valid_pts[:, 0]>=xmin, valid_pts[:, 0]<xmax),
            np.logical_and(valid_pts[:, 1]>=ymin, valid_pts[:, 1]<ymax)
        )

        bbox_pts_all_mask = np.logical_or(bbox_pts_all_mask, bbox_point_mask)

    return bbox_pts_all_mask

def main():
    traj    = 3
    frame   = 1800
    tred_sensor = "os1"
    twod_sensor = "cam0"
    indir = "/robodata/arthurz/Datasets/CODa"

    extr_calib_path = join(indir, "calibrations", str(traj), "calib_os1_to_cam0.yaml")
    intr_calib_path = join(indir, "calibrations", str(traj), "calib_cam0_intrinsics.yaml")

    tred_bbox_file  = set_filename_by_prefix("3d_bbox", tred_sensor, traj, frame)
    twod_bbox_file  = set_filename_by_prefix("2d_bbox", twod_sensor, traj, frame)
    pc_file         = set_filename_by_prefix("3d_raw", tred_sensor, traj, frame)
    img_file        = set_filename_by_prefix("2d_rect", twod_sensor, traj, frame)

    tred_bbox_path  = join(indir, "3d_bbox", tred_sensor, str(traj), tred_bbox_file)
    twod_bbox_path  = join(indir, "2d_bbox", twod_sensor, str(traj), twod_bbox_file)
    img_path        = join(indir, "2d_rect", twod_sensor, str(traj), img_file)
    pc_path         = join(indir, "3d_raw", tred_sensor, str(traj), pc_file)

    pc_np = read_bin(pc_path, keep_intensity=False)
    image           = cv2.imread(img_path)

    # These image pts 
    image_pts, pts_mask = project_3dto2d_points(pc_np, extr_calib_path, intr_calib_path, None)
    in_bounds = np.logical_and(
            np.logical_and(image_pts[:, 0]>=0, image_pts[:, 0]<1224),
            np.logical_and(image_pts[:, 1]>=0, image_pts[:, 1]<1024)
        )
    valid_point_mask = in_bounds & pts_mask
    valid_points    = image_pts[valid_point_mask, :]
    valid_3d_points = pc_np[valid_point_mask, :]

    print("valid_points ", valid_points.shape)
    for pt in valid_points:
        image = cv2.circle(image, (pt[0], pt[1]), radius=1, color=(0, 0, 255))

    anno_dict       = json.load(open(tred_bbox_path))
    bbox_coords     = project_3dto2d_bbox_image(anno_dict, extr_calib_path, intr_calib_path)

    tred_bbox_image = draw_2d_bbox(image, bbox_coords)
    cv2.imwrite("test2dbboxcalibration.png", tred_bbox_image)

    bbox_pts_all_mask = mask_points_in_bbox(bbox_coords, valid_points)
    # import pdb; pdb.set_trace()
    image           = cv2.imread(img_path)
    
    valid_points    = valid_points[bbox_pts_all_mask, :]
    valid_3d_points = valid_3d_points[bbox_pts_all_mask, :]
    # filtered_points = valid_points[0:5, :]
    for pt in valid_points:
        image = cv2.circle(image, (pt[0], pt[1]), radius=1, color=(0, 0, 255))
    cv2.imwrite("mask2dbboxcalibration.png", image)
    valid_3d_points.dump("bbox_pts")
    # import pdb; pdb.set_trace()

    # import plotly.graph_objects as go

    # # Create a list of 3D points
    # points = valid_3d_points

    # # Create a 3D scatter plot
    # fig = go.Figure(data=[go.Scatter3d(
    #     x=points[:,0],
    #     y=points[:,1],
    #     z=points[:,2],
    #     mode='markers',
    #     marker=dict(
    #         size=5,
    #         color='blue',
    #         opacity=0.8
    #     )
    # )])

    # # Set the axis labels
    # fig.update_layout(scene=dict(
    #     xaxis_title='X',
    #     yaxis_title='Y',
    #     zaxis_title='Z'
    # ))

    # # Show the plot
    # fig.show()

    

if __name__ == "__main__":
    main()

: 